# Regression Week 5: LASSO (coordinate descent)

In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

# Fire up graphlab create

Make sure you have the latest version of graphlab (>= 1.7)

In [161]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [162]:
sales = graphlab.SFrame('kc_house_data.gl/')
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before using it below
sales['floors'] = sales['floors'].astype(int) 

If we want to do any "feature engineering" like creating new features or adjusting existing ones we should do this directly using the SFrames as seen in the first notebook of Week 2. For this notebook, however, we will work with the existing features.

# Import useful functions from previous notebook

As in Week 2, we convert the SFrame into a 2D Numpy array. Copy and paste `get_num_data()` from the second notebook of Week 2.

In [163]:
import numpy as np # note this allows us to refer to numpy as np instead 

In [164]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1
    features = ['constant'] + features # this is how you combine two lists
    
    features_sframe = data_sframe[features]
    feature_matrix = features_sframe.to_numpy()
    
    output_sarray = data_sframe[output]
    output_array = output_sarray.to_numpy()
    
    return(feature_matrix, output_array)

Also, copy and paste the `predict_output()` function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [165]:
def predict_output(feature_matrix, weights):
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [166]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print X

[[  3.   5.   8.]
 [  4.  12.  15.]]


Numpy provides a shorthand for computing 2-norms of each column:

In [90]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print norms

[  5.  13.  17.]


To normalize, apply element-wise division:

In [91]:
print X / norms # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[ 0.6         0.38461538  0.47058824]
 [ 0.8         0.92307692  0.88235294]]


Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [218]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    normalized_feature_matrix = feature_matrix / norms
    return (normalized_feature_matrix, norms)

To test the function, run the following:

In [219]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print features
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print norms
# should print
# [5.  10.  15.]

[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]
[  5.  10.  15.]


# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [220]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

Don't forget to normalize features:

In [221]:
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix)

We assign some random set of initial weights and inspect the values of `ro[i]`:

In [222]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [223]:
prediction = predict_output(normalized_simple_feature_matrix, weights)

Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [225]:
def compute_ro(feature_i, output, prediction, weight_i):
    return sum(feature_i * (output-prediction + (weight_i*feature_i)))

num_features = len(normalized_simple_feature_matrix[0])

ros = []

for i in range(num_features):
    feature_i = normalized_simple_feature_matrix[:,i]
    weight_i = weights[i]
    ro = compute_ro(feature_i, output, prediction, weight_i)
    ros.append(ro)
    
for i in range(num_features):
    print "ro", i, "=", ros[i]

ro 0 = 79400300.0349
ro 1 = 87939470.773
ro 2 = 80966698.676


***QUIZ QUESTION: Recall that, whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight `w[i]` is set to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` would not set `w[1]` zero, but would set `w[2]` to zero, if we were to take a step in that coordinate?*** 

In [226]:
ro_1 = ros[1]
ro_2 = ros[2]

#-x/2 > ro_1 or ro_1 > x/2
#-x/2 <= ro2 <= x/2

l1_penalty_values = [1.4e8, 1.64e8, 1.73e8, 1.9e8, 2.3e8]
a = 1.4e8
b = 1.64e8
c = 1.73e8
d = 1.9e8
e = 2.3e8

def does_not_set_w1_to_zero_but_sets_w2_to_zero(l1_penalty, ro_1, ro_2):
    w1_to_zero = ro_1 >= -l1_penalty/2 and ro_1 <= l1_penalty/2
    w2_to_zero = ro_2 >= -l1_penalty/2 and ro_2 <= l1_penalty/2
    return not w1_to_zero and w2_to_zero

for i in range(len(l1_penalty_values)):
    if does_not_set_w1_to_zero_but_sets_w2_to_zero(l1_penalty_values[i], ro_1, ro_2):
        print l1_penalty_values[i]

164000000.0
173000000.0


161933396 < x <= 175878940

***QUIZ QUESTION: What range of values of `l1_penalty` would set both `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate?*** 

In [227]:
#-x/2 <= ro1 <= x/2
#-x/2 <= ro2 <= x/2

def sets_both_w1_and_w2_to_zero(l1_penalty, ro_1, ro_2):
    w1_to_zero = ro_1 >= -l1_penalty/2 and ro_1 <= l1_penalty/2
    w2_to_zero = ro_2 >= -l1_penalty/2 and ro_2 <= l1_penalty/2
    return w1_to_zero and w2_to_zero

for i in range(len(l1_penalty_values)):
    if sets_both_w1_and_w2_to_zero(l1_penalty_values[i], ro_1, ro_2):
        print l1_penalty_values[i]

190000000.0
230000000.0


So we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [228]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    feature_i = feature_matrix[:,i]
    weight_i = weights[i]
    ro_i = compute_ro(feature_i, output, prediction, weight_i)

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2.
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2.
    else:
        new_weight_i = 0.
    
    return new_weight_i

To test the function, run the following cell:

In [229]:
# should print 0.425558846691
import math
print lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.425558846691


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [230]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    weights = initial_weights
    max_weight_change = tolerance
    
    while max_weight_change >= tolerance:
        max_weight_change = 0
        weight_changes = []
        
        for i in range(len(weights)):
            old_weight_i = weights[i]
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
            weight_change = abs(old_weight_i - weights[i])
            
            #print "old weight = ", old_weight_i
            #print "new weight = ", weights[i]
            #print "weight change = ", weight_change
            
            weight_changes.append(weight_change)
            
        max_weight_change = max(weight_changes)
        
        print "max weight change for iteration = ", max_weight_change         
    
    return weights

Using the following parameters, learn the weights on the sales dataset. 

In [231]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`

In [232]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

Then, run your implementation of LASSO coordinate descent:

In [233]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

max weight change for iteration =  79400304.6581
max weight change for iteration =  9138168.3665
max weight change for iteration =  8194809.40937
max weight change for iteration =  6598905.03827
max weight change for iteration =  5522173.1943
max weight change for iteration =  4621129.81031
max weight change for iteration =  3867108.10625
max weight change for iteration =  3236118.81061
max weight change for iteration =  2708086.93955
max weight change for iteration =  2266213.1094
max weight change for iteration =  1896439.06265
max weight change for iteration =  1587000.40319
max weight change for iteration =  1328052.31095
max weight change for iteration =  1111356.328
max weight change for iteration =  930018.251242
max weight change for iteration =  778268.79269
max weight change for iteration =  651280.029038
max weight change for iteration =  545011.800818
max weight change for iteration =  456083.174344
max weight change for iteration =  381664.876995
max weight change for iter

***QUIZ QUESTION: What is the RSS of the learned model on the normalized dataset?***

In [234]:
prediction = predict_output(normalized_simple_feature_matrix, weights)
errors = prediction - output
rss = sum(errors * errors)

print rss

1.63049248148e+15


***QUIZ QUESTION: Which features had weight zero at convergence?***

In [236]:
for i in range(len(weights)):
    if weights[i] == 0.0:
        print simple_features[i-1]

bedrooms


# Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [237]:
train_data,test_data = sales.random_split(.8,seed=0)

Let us consider the following set of features.

In [238]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

First, create a normalized feature matrix from the TRAINING data with these features.  (Make you store the norms for the normalization, since we'll use them later)

In [239]:
(all_feature_matrix, output) = get_numpy_data(train_data, all_features, my_output)
(normalized_all_feature_matrix, all_norms) = normalize_features(all_feature_matrix)

First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [268]:
initial_weights = np.zeros(len(all_features)+1)
l1_penalty = 1e7
tolerance = 1.0

weights1e7 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, 
                                               output,
                                               initial_weights,
                                               l1_penalty,
                                               tolerance)

max weight change for iteration =  71114625.7528
max weight change for iteration =  5024356.36869
max weight change for iteration =  5012324.91395
max weight change for iteration =  5000676.42086
max weight change for iteration =  4991243.17314
max weight change for iteration =  4984361.34335
max weight change for iteration =  4979504.92921
max weight change for iteration =  4973806.56526
max weight change for iteration =  2879222.99938
max weight change for iteration =  2706150.76622
max weight change for iteration =  2532295.64864
max weight change for iteration =  2368316.57782
max weight change for iteration =  2213124.35067
max weight change for iteration =  1552443.60013
max weight change for iteration =  507838.033159
max weight change for iteration =  438814.370591
max weight change for iteration =  371479.622608
max weight change for iteration =  313415.105581
max weight change for iteration =  264244.832139
max weight change for iteration =  222753.8608
max weight change for 

***QUIZ QUESTION: What features had non-zero weight in this case?***

In [269]:
for i in range(len(weights1e7)):
    if i == 0:
        print "constant:", weights1e7[i]
    else:
        print all_features[i-1], ":", weights1e7[i]
print ''

for i in range(len(weights1e7)):
    if weights1e7[i] != 0.0:
        if i == 0:
            print "constant"
        else:
            print all_features[i-1]

constant: 24429600.6093
bedrooms : 0.0
bathrooms : 0.0
sqft_living : 48389174.3523
sqft_lot : 0.0
floors : 0.0
waterfront : 3317511.16272
view : 7329961.9849
condition : 0.0
grade : 0.0
sqft_above : 0.0
sqft_basement : 0.0
yr_built : 0.0
yr_renovated : 0.0

constant
sqft_living
waterfront
view


Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [270]:
my_output = 'price'
initial_weights = np.zeros(len(all_features)+1)
l1_penalty = 1e8
tolerance = 1.0

weights1e8 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, 
                                               output,
                                               initial_weights,
                                               l1_penalty,
                                               tolerance)

max weight change for iteration =  71114625.7528
max weight change for iteration =  0.0


***QUIZ QUESTION: What features had non-zero weight in this case?***

In [271]:
for i in range(len(weights1e8)):
    if i == 0:
        print "constant:", weights1e8[i]
    else:
        print all_features[i-1], ":", weights1e8[i]
print ''

for i in range(len(weights1e8)):
    if weights1e8[i] != 0.0:
        if i == 0:
            print "constant"
        else:
            print all_features[i-1]

constant: 71114625.7528
bedrooms : 0.0
bathrooms : 0.0
sqft_living : 0.0
sqft_lot : 0.0
floors : 0.0
waterfront : 0.0
view : 0.0
condition : 0.0
grade : 0.0
sqft_above : 0.0
sqft_basement : 0.0
yr_built : 0.0
yr_renovated : 0.0

constant


Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.  Call resulting weights `weights1e4`, you will need them later.  (This case will take quite a bit longer to converge than the others above.)

In [272]:
my_output = 'price'
initial_weights = np.zeros(len(all_features)+1)
l1_penalty = 1e4
tolerance = 5e5

weights1e4 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, 
                                               output,
                                               initial_weights,
                                               l1_penalty,
                                               tolerance)

max weight change for iteration =  71114625.7528
max weight change for iteration =  8464015.94738
max weight change for iteration =  6818451.35869
max weight change for iteration =  5570165.21901
max weight change for iteration =  4559838.46759
max weight change for iteration =  3732056.69715
max weight change for iteration =  3053211.61353
max weight change for iteration =  2497517.43558
max weight change for iteration =  2043868.97325
max weight change for iteration =  1674717.70906
max weight change for iteration =  1375420.75692
max weight change for iteration =  1296621.8143
max weight change for iteration =  1257340.9361
max weight change for iteration =  1228271.24819
max weight change for iteration =  1206854.36471
max weight change for iteration =  1191059.40598
max weight change for iteration =  1179030.77018
max weight change for iteration =  1169777.43345
max weight change for iteration =  1162340.67589
max weight change for iteration =  1155980.52901
max weight change for 

***QUIZ QUESTION: What features had non-zero weight in this case?***

In [254]:
for i in range(len(weights1e4)):
    if i == 0:
        print "constant:", weights1e4[i]
    else:
        print all_features[i-1], ":", weights1e4[i]
print ''

for i in range(len(weights1e4)):
    if weights1e4[i] != 0.0:
        if i == 0:
            print "constant"
        else:
            print all_features[i-1]

constant: 77779073.9127
bedrooms : -22884012.2502
bathrooms : 15348487.0809
sqft_living : 92166869.6988
sqft_lot : -2139328.08243
floors : -8818455.54409
waterfront : 6494209.73311
view : 7065162.05053
condition : 4119079.21007
grade : 18436483.5262
sqft_above : -14566678.5451
sqft_basement : -5528348.75179
yr_built : -83591746.2073
yr_renovated : 2784276.46013

constant
bedrooms
bathrooms
sqft_living
sqft_lot
floors
waterfront
view
condition
grade
sqft_above
sqft_basement
yr_built
yr_renovated


## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [265]:
normalized_weights1e7 = weights1e7 / all_norms
normalized_weights1e8 = weights1e8 / all_norms
normalized_weights1e4 = weights1e4 / all_norms

To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print normalized_weights1e7[3]
```
should return 161.31745624837794.

In [266]:
print normalized_weights1e7[3]

161.317456248


## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [259]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, my_output)

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [261]:
prediction = predict_output(test_feature_matrix, normalized_weights1e7)
errors = prediction - test_output
weights1e7_rss = sum(errors * errors)

print weights1e7_rss

2.75962079909e+14


In [262]:
prediction = predict_output(test_feature_matrix, normalized_weights1e8)
errors = prediction - test_output
weights1e8_rss = sum(errors * errors)

print weights1e8_rss

5.37166150034e+14


In [263]:
prediction = predict_output(test_feature_matrix, normalized_weights1e4)
errors = prediction - test_output
weights1e4_rss = sum(errors * errors)

print weights1e4_rss

2.2778100476e+14


***QUIZ QUESTION: Which model performed best on the test data?***

In [264]:
rsss = [weights1e7_rss, weights1e8_rss, weights1e4_rss]
min_rss = min(rsss)
if min_rss == weights1e7_rss:
    print "Model trained with 1e7"
elif min_rss == weights1e8_rss:
    print "Model trained with 1e8"
else:
    print "Model trained with 1e4"

Model trained with 1e4
